In [1]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/image-matching-challenge-2025/sample_submission.csv
/kaggle/input/image-matching-challenge-2025/train_thresholds.csv
/kaggle/input/image-matching-challenge-2025/train_labels.csv
/kaggle/input/image-matching-challenge-2025/test/ETs/another_et_another_et004.png
/kaggle/input/image-matching-challenge-2025/test/ETs/outliers_out_et003.png
/kaggle/input/image-matching-challenge-2025/test/ETs/another_et_another_et006.png
/kaggle/input/image-matching-challenge-2025/test/ETs/et_et004.png
/kaggle/input/image-matching-challenge-2025/test/ETs/et_et002.png
/kaggle/input/image-matching-challenge-2025/test/ETs/another_et_another_et009.png
/kaggle/input/image-matching-challenge-2025/test/ETs/outliers_out_et001.png
/kaggle/input/image-matching-challenge-2025/test/ETs/et_et006.png
/kaggle/input/image-matching-challenge-2025/test/ETs/another_et_another_et005.png
/kaggle/input/image-matching-challenge-2025/test/ETs/another_et_another_et008.png
/kaggle/input/image-matching-challenge-2025/test

In [2]:
import sys
import os
import subprocess
from pathlib import Path
import shutil
import glob

# --- CONFIGURATION ---
INPUT_ROOT = Path("/kaggle/input")
DATASET_NAME = "imc-2025-deps" 

# 1. Locate Wheels
wheels_path = None
candidates = [
    INPUT_ROOT / DATASET_NAME / "offline_wheels",
    INPUT_ROOT / "offline_wheels",
]
candidates.extend(list(INPUT_ROOT.rglob("offline_wheels")))

for cand in candidates:
    if cand.exists() and cand.is_dir():
        wheels_path = cand
        break

if not wheels_path:
    # Fallback extraction
    zip_found = list(INPUT_ROOT.rglob("my_offline_dependencies.zip"))
    if zip_found:
        print(f"Extracting zip from {zip_found[0]}...")
        EXTRACT_PATH = Path("/kaggle/working/offline_deps")
        EXTRACT_PATH.mkdir(exist_ok=True, parents=True)
        subprocess.run(["unzip", "-q", "-o", str(zip_found[0]), "-d", str(EXTRACT_PATH)], check=True)
        wheels_path = EXTRACT_PATH / "offline_wheels"
    else:
        raise FileNotFoundError("CRITICAL: Wheels not found.")

print(f"✅ Found wheels at: {wheels_path}")

# --- CRITICAL FIX: CLEAN AND PREPARE ---
print("Copying wheels to temp location...")
INSTALL_DIR = Path("/kaggle/working/temp_install")
if INSTALL_DIR.exists(): shutil.rmtree(INSTALL_DIR)
shutil.copytree(wheels_path, INSTALL_DIR)

print("Removing conflicting wheels...")
# ADDED 'pillow*' to this list to fix the _Ink error
# We want to use the system Pillow which works with system Torchvision
dangerous_patterns = ["torch*", "torchvision*", "numpy*", "nvidia*", "pillow*", "Pillow*"]

count = 0
for pattern in dangerous_patterns:
    for f in INSTALL_DIR.glob(pattern):
        print(f"Removing {f.name} (Using system version)")
        f.unlink()
        count += 1
print(f"Deleted {count} conflicting wheels.")

# --- THE NUCLEAR INSTALL ---
# We find ALL remaining wheels and install them with --no-deps
wheel_files = [str(f) for f in INSTALL_DIR.glob("*.whl")]

print(f"Force installing {len(wheel_files)} libraries...")

subprocess.run(
    [sys.executable, "-m", "pip", "install", "--no-index", "--no-deps"] + wheel_files,
    check=True
)

# Cleanup
shutil.rmtree(INSTALL_DIR)
if Path("/kaggle/working/offline_deps").exists():
    shutil.rmtree("/kaggle/working/offline_deps")

print("\n✅ Installation Complete.")

✅ Found wheels at: /kaggle/input/imc-2025-deps/offline_wheels
Copying wheels to temp location...
Removing conflicting wheels...
Removing torchvision-0.24.1-cp311-cp311-manylinux_2_28_x86_64.whl (Using system version)
Removing torch-2.9.1-cp311-cp311-manylinux_2_28_x86_64.whl (Using system version)
Removing numpy-2.3.5-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (Using system version)
Removing numpy-2.2.6-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (Using system version)
Removing nvidia_cusparselt_cu12-0.7.1-py3-none-manylinux2014_x86_64.whl (Using system version)
Removing nvidia_nvtx_cu12-12.8.90-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (Using system version)
Removing nvidia_cufft_cu12-11.3.3.83-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (Using system version)
Removing nvidia_cublas_cu12-12.8.4.1-py3-none-manylinux_2_27_x86_64.whl (Using system version)
Removing nvidia_cusolver_cu12-11.7.3.90-py3-none-manylinux_2_27_x86_64.whl 

In [3]:
import sys
import os
from pathlib import Path
import torch
import numpy as np
import cv2
from tqdm import tqdm
import shutil
import h5py
import sqlite3
from types import ModuleType

# --- MONKEY PATCH START ---
MAX_IMAGE_ID = 2**31 - 1

class COLMAPDatabase(sqlite3.Connection):
    @staticmethod
    def connect(database_path):
        return sqlite3.connect(database_path, factory=COLMAPDatabase)

    def __init__(self, *args, **kwargs):
        super(COLMAPDatabase, self).__init__(*args, **kwargs)
        self.create_tables()

    def create_tables(self):
        self.create_cameras_table()
        self.create_images_table()
        self.create_keypoints_table()
        self.create_descriptors_table()
        self.create_matches_table()
        self.create_two_view_geometries_table()

    def create_cameras_table(self):
        self.execute("CREATE TABLE IF NOT EXISTS cameras (camera_id INTEGER PRIMARY KEY AUTOINCREMENT, model INTEGER NOT NULL, width INTEGER NOT NULL, height INTEGER NOT NULL, params BLOB, prior_focal_length INTEGER NOT NULL)")

    def create_images_table(self):
        self.execute("CREATE TABLE IF NOT EXISTS images (image_id INTEGER PRIMARY KEY AUTOINCREMENT, name TEXT NOT NULL UNIQUE, camera_id INTEGER NOT NULL, prior_qw REAL, prior_qx REAL, prior_qy REAL, prior_qz REAL, prior_tx REAL, prior_ty REAL, prior_tz REAL, CONSTRAINT image_id_check CHECK(image_id >= 0 AND image_id < {}))".format(MAX_IMAGE_ID))

    def create_keypoints_table(self):
        self.execute("CREATE TABLE IF NOT EXISTS keypoints (image_id INTEGER PRIMARY KEY, rows INTEGER NOT NULL, cols INTEGER NOT NULL, data BLOB, FOREIGN KEY(image_id) REFERENCES images(image_id) ON DELETE CASCADE)")

    def create_descriptors_table(self):
        self.execute("CREATE TABLE IF NOT EXISTS descriptors (image_id INTEGER PRIMARY KEY, rows INTEGER NOT NULL, cols INTEGER NOT NULL, data BLOB, FOREIGN KEY(image_id) REFERENCES images(image_id) ON DELETE CASCADE)")

    def create_matches_table(self):
        self.execute("CREATE TABLE IF NOT EXISTS matches (pair_id INTEGER PRIMARY KEY, rows INTEGER NOT NULL, cols INTEGER NOT NULL, data BLOB)")

    def create_two_view_geometries_table(self):
        self.execute("CREATE TABLE IF NOT EXISTS two_view_geometries (pair_id INTEGER PRIMARY KEY, rows INTEGER NOT NULL, cols INTEGER NOT NULL, data BLOB, config INTEGER NOT NULL, F BLOB, E BLOB, H BLOB, qvec BLOB, tvec BLOB)")

    def add_camera(self, model, width, height, params, prior_focal_length=False, camera_id=None):
        params = np.asarray(params, np.float64)
        cursor = self.execute("INSERT INTO cameras VALUES (?, ?, ?, ?, ?, ?)", (camera_id, model, width, height, params.tobytes(), prior_focal_length))
        return cursor.lastrowid

    def add_image(self, name, camera_id, prior_q=np.full(4, np.nan), prior_t=np.full(3, np.nan), image_id=None):
        cursor = self.execute("INSERT INTO images VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", (image_id, name, camera_id, prior_q[0], prior_q[1], prior_q[2], prior_q[3], prior_t[0], prior_t[1], prior_t[2]))
        return cursor.lastrowid

    def add_keypoints(self, image_id, keypoints):
        assert(len(keypoints.shape) == 2)
        assert(keypoints.shape[1] in [2, 4, 6])
        keypoints = np.asarray(keypoints, np.float32)
        self.execute("INSERT INTO keypoints VALUES (?, ?, ?, ?)", (image_id, keypoints.shape[0], keypoints.shape[1], keypoints.tobytes()))

    def add_descriptors(self, image_id, descriptors):
        descriptors = np.ascontiguousarray(descriptors, np.uint8)
        self.execute("INSERT INTO descriptors VALUES (?, ?, ?, ?)", (image_id, descriptors.shape[0], descriptors.shape[1], descriptors.tobytes()))

    def add_matches(self, image_id1, image_id2, matches):
        pair_id = image_ids_to_pair_id(image_id1, image_id2)
        if image_id1 > image_id2:
            matches = matches[:, [1, 0]]
        matches = matches.astype(np.uint32)
        self.execute("INSERT INTO matches VALUES (?, ?, ?, ?)", (pair_id, matches.shape[0], matches.shape[1], matches.tobytes()))
    
    # Aliases used by HLOC
    write_matches = add_matches
    write_keypoints = add_keypoints
    write_descriptors = add_descriptors

def blob_to_array(blob, dtype, shape=(-1,)):
    return np.frombuffer(blob, dtype).reshape(shape)

def array_to_blob(array):
    return array.tobytes()

def pair_id_to_image_ids(pair_id):
    image_id2 = pair_id % MAX_IMAGE_ID
    image_id1 = (pair_id - image_id2) // MAX_IMAGE_ID
    return image_id1, image_id2

def image_ids_to_pair_id(image_id1, image_id2):
    if image_id1 > image_id2:
        image_id1, image_id2 = image_id2, image_id1
    return image_id1 * MAX_IMAGE_ID + image_id2

def get_image_ids(database_path):
    db = COLMAPDatabase.connect(database_path)
    image_ids = {}
    for name, image_id in db.execute("SELECT name, image_id FROM images"):
        image_ids[name] = image_id
    db.close()
    return image_ids

fake_db_module = ModuleType("hloc.utils.database")
fake_db_module.COLMAPDatabase = COLMAPDatabase
fake_db_module.pair_id_to_image_ids = pair_id_to_image_ids
fake_db_module.image_ids_to_pair_id = image_ids_to_pair_id
fake_db_module.get_image_ids = get_image_ids
fake_db_module.blob_to_array = blob_to_array
fake_db_module.array_to_blob = array_to_blob

sys.modules["hloc.utils.database"] = fake_db_module
import hloc.utils
hloc.utils.database = fake_db_module
# --- MONKEY PATCH END ---

import pycolmap
from hloc import reconstruction, pairs_from_retrieval
import romatch 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Find weights
weights_dir = None
candidates = list(Path("/kaggle/input").rglob("weights"))
for cand in candidates:
    if (cand / "roma").exists():
        weights_dir = cand
        break
if not weights_dir:
    if Path("/kaggle/working/offline_deps/weights").exists():
        weights_dir = Path("/kaggle/working/offline_deps/weights")
    else:
        raise FileNotFoundError("CRITICAL: Could not find 'weights' folder.")

print(f"✅ Found weights at: {weights_dir}")

OUTPUT_DIR = Path("/kaggle/working/outputs")
if OUTPUT_DIR.exists(): shutil.rmtree(OUTPUT_DIR)
OUTPUT_DIR.mkdir(parents=True)

DATA_ROOT = Path("/kaggle/input/image-matching-challenge-2025")
TEST_DIR = DATA_ROOT / "test"

CONF = {
    'roma_weights': weights_dir / "roma" / "roma_outdoor.pth",
    'dino_weights': weights_dir / "dinov2" / "dinov2_vitl14.pth",
    'max_keypoints': 5000,  
    'match_threshold': 0.2,
}

Using device: cuda
✅ Found weights at: /kaggle/input/imc-2025-deps/weights


In [4]:
def load_models():
    print("Loading Models...")
    
    # 1. Load DINOv2 State Dict
    print(f"Loading DINOv2 weights from {CONF['dino_weights']}...")
    dino_state_dict = torch.load(str(CONF['dino_weights']), map_location=device)
    
    # 2. Load RoMa State Dict
    print(f"Loading RoMa weights from {CONF['roma_weights']}...")
    roma_state_dict = torch.load(str(CONF['roma_weights']), map_location=device)
    
    # 3. Initialize RoMa
    # CRITICAL FIX: Set use_custom_corr=False to use pure PyTorch mode
    roma_model = romatch.roma_outdoor(
        device=device, 
        weights=roma_state_dict,       
        dinov2_weights=dino_state_dict,
        use_custom_corr=False # <--- THIS PREVENTS THE ERROR
    )
    print("✅ RoMa loaded successfully (PyTorch Backend).")
    
    return roma_model

# Initialize
matcher_model = load_models()

Loading Models...
Loading DINOv2 weights from /kaggle/input/imc-2025-deps/weights/dinov2/dinov2_vitl14.pth...
Loading RoMa weights from /kaggle/input/imc-2025-deps/weights/roma/roma_outdoor.pth...


2025-11-24 10:04:17.360 | INFO     | romatch.models.model_zoo.roma_models:roma_model:61 - Using coarse resolution (560, 560), and upsample res (864, 864)


✅ RoMa loaded successfully (PyTorch Backend).


In [5]:
def get_matches(model, img1_path, img2_path):
    """
    Runs RoMa to find dense matches between two images.
    Returns: Nx2 arrays for Keypoints in Image 1 and Image 2
    """
    try:
        # 1. Get Image Dimensions (Needed for coordinate normalization)
        # We use cv2 to quickly read metadata
        im1 = cv2.imread(str(img1_path))
        im2 = cv2.imread(str(img2_path))
        h1, w1 = im1.shape[:2]
        h2, w2 = im2.shape[:2]

        # RoMa handles loading and resizing internally for matching
        with torch.no_grad():
            # 2. Run the dense matching
            # Ensure paths are strings
            warp, certainty = model.match(str(img1_path), str(img2_path), device=device)
            
            # 3. Sample matches 
            # RoMa produces dense warps. We sample the best ones.
            matches, certainty = model.sample(
                warp, certainty, 
                num=CONF['max_keypoints']
            )
            
            # 4. Convert to pixel coordinates
            # Pass integers (H, W) instead of paths
            kpts1, kpts2 = model.to_pixel_coordinates(matches, h1, w1, h2, w2)
            
            return kpts1.cpu().numpy(), kpts2.cpu().numpy()
            
    except Exception as e:
        print(f"Matching failed for {img1_path.name} - {img2_path.name}: {e}")
        return np.empty((0, 2)), np.empty((0, 2))

In [6]:
import pycolmap
from hloc.utils import database
import shutil
import h5py
import numpy as np
from pathlib import Path
import os

def quaternion_to_rotation_matrix(qvec):
    """Convert quaternion [w, x, y, z] to 3x3 rotation matrix."""
    return pycolmap.qvec_to_rotmat(qvec)

def run_sfm_pipeline(dataset_name, image_dir, matches_dict):
    """
    Takes raw matches, formats them for HLOC, and runs SfM.
    """
    output_path = OUTPUT_DIR / dataset_name
    output_path.mkdir(exist_ok=True, parents=True)
    
    features_h5 = output_path / "features.h5"
    matches_h5 = output_path / "matches.h5"
    pairs_txt = output_path / "pairs.txt"
    sfm_dir = output_path / "sfm"
    database_path = output_path / "database.db"
    
    if sfm_dir.exists(): shutil.rmtree(sfm_dir)
    sfm_dir.mkdir(exist_ok=True)
    
    # --- 1. Quantize Keypoints ---
    keypoints_map = {} 
    unique_kpts = {} 
    
    print(f"Quantizing keypoints for {len(matches_dict)} pairs...")
    
    for (name0, name1), (mkpts0, mkpts1) in matches_dict.items():
        for name, kpts in [(name0, mkpts0), (name1, mkpts1)]:
            if name not in unique_kpts:
                unique_kpts[name] = []
                keypoints_map[name] = {}
            
            for (x, y) in kpts:
                # Integer rounding for robust track linking
                pt_key = (int(round(x)), int(round(y)))
                
                if pt_key not in keypoints_map[name]:
                    idx = len(unique_kpts[name])
                    unique_kpts[name].append([x, y]) 
                    keypoints_map[name][pt_key] = idx

    # --- 2. Write features.h5 ---
    print("Writing features to HDF5...")
    with h5py.File(features_h5, 'w') as f_feat:
        for name, kpts in unique_kpts.items():
            kpts_arr = np.array(kpts, dtype=np.float32)
            grp = f_feat.create_group(name)
            grp.create_dataset('keypoints', data=kpts_arr)
            grp.create_dataset('descriptors', data=np.zeros((len(kpts), 128), dtype=np.float32))
            grp.create_dataset('scores', data=np.ones((len(kpts),), dtype=np.float32))

    # --- 3. Write matches.h5 ---
    print("Writing matches to HDF5...")
    raw_pairs = []
    with h5py.File(matches_h5, 'w') as f_match:
        for (name0, name1), (mkpts0, mkpts1) in matches_dict.items():
            id0 = [keypoints_map[name0].get((int(round(x)), int(round(y)))) for x, y in mkpts0]
            id1 = [keypoints_map[name1].get((int(round(x)), int(round(y)))) for x, y in mkpts1]
            
            valid_indices = [i for i, (i0, i1) in enumerate(zip(id0, id1)) if i0 is not None and i1 is not None]
            if not valid_indices: continue
            
            n_kpts0 = len(unique_kpts[name0])
            matches_array = np.full(n_kpts0, -1, dtype=np.int32)
            
            for i in valid_indices:
                idx0 = id0[i]
                idx1 = id1[i]
                matches_array[idx0] = idx1
            
            grp = f_match.require_group(name0)
            grp_pair = grp.require_group(name1)
            grp_pair.create_dataset("matches0", data=matches_array)
            grp_pair.create_dataset("matching_scores0", data=np.ones(n_kpts0, dtype=np.float32))
            
            raw_pairs.append((name0, name1))

    # --- 4. Run Reconstruction ---
    try:
        if database_path.exists(): database_path.unlink()
        reconstruction.create_empty_db(database_path)
        
        # FIX 1: Use AUTO mode
        reconstruction.import_images(image_dir, database_path, pycolmap.CameraMode.AUTO)
        
        # FIX 2: Filter pairs
        image_ids = database.get_image_ids(database_path)
        valid_pairs = []
        for p1, p2 in raw_pairs:
            if p1 in image_ids and p2 in image_ids:
                valid_pairs.append((p1, p2))
        
        # Write valid pairs
        with open(pairs_txt, 'w') as f:
            for p1, p2 in valid_pairs:
                f.write(f"{p1} {p2}\n")

        # Continue Import
        db = database.COLMAPDatabase.connect(database_path)
        reconstruction.import_features(image_ids, db, features_h5)
        reconstruction.import_matches(image_ids, db, pairs_txt, matches_h5, min_match_score=None, skip_geometric_verification=False)
        db.commit()
        db.close()
        
        # 5. Geometric Verification
        print("Running geometric verification...")
        options = pycolmap.TwoViewGeometryOptions()
        try:
            options.ransac.max_error = 4.0 
            options.ransac.min_inlier_ratio = 0.1
        except: pass

        pycolmap.verify_matches(str(database_path), str(pairs_txt), options=options)
        
        # 6. Incremental Mapping (FIXED OPTIONS TYPE)
        print("Running incremental mapper...")
        
        # We use IncrementalPipelineOptions, NOT IncrementalMapperOptions directly
        pipeline_opts = pycolmap.IncrementalPipelineOptions()
        
        # Access the inner mapper options to set constraints
        pipeline_opts.min_num_matches = 15
        pipeline_opts.mapper.init_min_num_inliers = 25
        
        maps = pycolmap.incremental_mapping(
            database_path=str(database_path),
            image_path=str(image_dir),
            output_path=str(sfm_dir),
            options=pipeline_opts # <--- Passed correct type
        )
        
        if not maps:
            print(f"⚠️ No reconstruction found for {dataset_name}")
            return None
        
        best_model = maps[0]
        print(f"✅ Best reconstruction: {len(best_model.images)} images registered")
        return best_model
        
    except Exception as e:
        print(f"❌ SfM Failed for {dataset_name}: {e}")
        import traceback
        traceback.print_exc()
        return None

In [7]:
def cleanup_disk(dataset_name):
    folder = OUTPUT_DIR / dataset_name
    if folder.exists():
        shutil.rmtree(folder)
    print(f"Deleted {folder} to free up disk space.")

def qvec2rotmat(qvec):
    w, x, y, z = qvec
    return np.array([
        [1 - 2*y*y - 2*z*z, 2*x*y - 2*z*w, 2*x*z + 2*y*w],
        [2*x*y + 2*z*w, 1 - 2*x*x - 2*z*z, 2*y*z - 2*x*w],
        [2*x*z - 2*y*w, 2*y*z + 2*x*w, 1 - 2*x*x - 2*y*y]
    ])

def process_all_datasets():
    submission_data = []
    if not TEST_DIR.exists(): return []
    
    datasets = [x for x in TEST_DIR.iterdir() if x.is_dir()]
    
    for ds_path in datasets:
        ds_name = ds_path.name
        print(f"\n=== Processing {ds_name} ===")
        
        images = sorted([x.name for x in ds_path.glob("*.png")])
        if len(images) < 3: continue
        
        # Matching Logic (Simplified for debug)
        pairs = []
        if len(images) < 300:
            print("Small dataset: Exhaustive matching.")
            for i in range(len(images)):
                for j in range(i + 1, len(images)):
                    pairs.append((images[i], images[j]))
        else:
            print("Large dataset: Sliding Window.")
            MAX_PAIRS = 3000
            for i in range(len(images)):
                for j in range(i + 1, min(i + 20, len(images))): 
                    pairs.append((images[i], images[j]))
            if len(pairs) > MAX_PAIRS:
                import random
                random.seed(42)
                pairs = random.sample(pairs, MAX_PAIRS)

        print(f"Matching {len(pairs)} pairs...")
        matches_dict = {}
        for name0, name1 in tqdm(pairs):
            kp0, kp1 = get_matches(matcher_model, ds_path/name0, ds_path/name1)
            if len(kp0) > 15:
                matches_dict[(name0, name1)] = (kp0, kp1)
        
        # Run Reconstruction
        model = None
        if len(matches_dict) > 0:
            model = run_sfm_pipeline(ds_name, ds_path, matches_dict)
            
        # --- DEBUG EXTRACTION START ---
        registered_images = set()
        if model:
            print(f"DEBUG: Model returned with {len(model.images)} images.")
            
            # Iterate over images
            for img_id, img_data in model.images.items():
                img_name = img_data.name
                registered_images.add(img_name)
                
                print(f"DEBUG: extracting {img_name}...")
                
                try:
                    # Attempt to extract pose
                    if hasattr(img_data, 'cam_from_world'):
                        pose = img_data.cam_from_world
                        if callable(pose): pose = pose()
                        rot = pose.rotation
                        if callable(rot): rot = rot()
                        qvec = rot.quat
                        tvec = pose.translation
                    else:
                        qvec = img_data.qvec
                        tvec = img_data.tvec
                    
                    # Debug the values
                    # print(f"  -> qvec: {qvec}, tvec: {tvec}")
                    
                    # Math
                    R = qvec2rotmat(qvec)
                    R_str = ";".join(map(str, R.flatten()))
                    T_str = ";".join(map(str, tvec))
                    
                    # Append
                    submission_data.append({
                        'dataset': ds_name,
                        'scene': 'scene1',
                        'image': img_name,
                        'rotation_matrix': R_str,
                        'translation_vector': T_str
                    })
                    print(f"  -> Success!")
                    
                except Exception as e:
                    print(f"  -> FAIL: {e}")
        else:
            print("DEBUG: Model was None.")
            
        # --- DEBUG EXTRACTION END ---

        # Handle Outliers
        count_outliers = 0
        for img in images:
            if img not in registered_images:
                submission_data.append({'dataset': ds_name, 'scene': 'scene1', 'image': img, 'rotation_matrix': np.nan, 'translation_vector': np.nan})
                count_outliers += 1
        print(f"DEBUG: Added {count_outliers} outliers.")
        
        cleanup_disk(ds_name)
        import gc
        gc.collect()
        torch.cuda.empty_cache()
            
    return submission_data

# Run pipeline
results = process_all_datasets()


=== Processing ETs ===
Small dataset: Exhaustive matching.
Matching 231 pairs...


100%|██████████| 231/231 [05:43<00:00,  1.48s/it]


Quantizing keypoints for 231 pairs...
Writing features to HDF5...
Writing matches to HDF5...


[2025/11/24 10:10:28 hloc INFO] Creating an empty database...
[2025/11/24 10:10:28 hloc INFO] Importing images into the database...
E20251124 10:10:28.779914 136400352617600 images.cc:56] LICENSE.txt BITMAP_ERROR: Failed to read the image file format.
[2025/11/24 10:10:28 hloc INFO] Importing features into the database...
100%|██████████| 22/22 [00:00<00:00, 44.65it/s]
[2025/11/24 10:10:29 hloc INFO] Importing matches into the database...
100%|██████████| 231/231 [00:00<00:00, 689.73it/s]


Running geometric verification...


...

Running geometric verification...


I20251124 11:00:03.993168 136394261390912 misc.cc:44] 
Feature matching & geometric verification
I20251124 11:00:03.993530 136394269783616 sift.cc:1452] Creating SIFT CPU feature matcher
I20251124 11:00:03.993647 136394320107072 sift.cc:1452] Creating SIFT CPU feature matcher
I20251124 11:00:03.993725 136394252998208 sift.cc:1452] Creating SIFT CPU feature matcher
I20251124 11:00:03.993780 136383456867904 sift.cc:1452] Creating SIFT CPU feature matcher
I20251124 11:00:03.993810 136394261390912 pairing.cc:810] Importing image pairs...
I20251124 11:00:03.994906 136394261390912 pairing.cc:843] Processing block [1/2]
I20251124 11:10:44.339073 136394261390912 feature_matching.cc:117] in 640.344s
I20251124 11:10:44.339206 136394261390912 pairing.cc:843] Processing block [2/2]
I20251124 11:11:08.635921 136394261390912 feature_matching.cc:117] in 24.297s
I20251124 11:11:08.635965 136394261390912 timer.cc:90] Elapsed time: 11.077 [minutes]


Running incremental mapper...


I20251124 11:11:09.315026 136400352617600 incremental_pipeline.cc:264] Loading database
I20251124 11:11:09.316687 136400352617600 database_cache.cc:67] Loading rigs...
I20251124 11:11:09.316778 136400352617600 database_cache.cc:77]  51 in 0.000s
I20251124 11:11:09.316804 136400352617600 database_cache.cc:85] Loading cameras...
I20251124 11:11:09.316924 136400352617600 database_cache.cc:103]  51 in 0.000s
I20251124 11:11:09.316939 136400352617600 database_cache.cc:111] Loading frames...
I20251124 11:11:09.317122 136400352617600 database_cache.cc:126]  51 in 0.000s
I20251124 11:11:09.317144 136400352617600 database_cache.cc:134] Loading matches...
I20251124 11:11:09.335047 136400352617600 database_cache.cc:139]  1275 in 0.018s
I20251124 11:11:09.335074 136400352617600 database_cache.cc:155] Loading images...
I20251124 11:11:09.715572 136400352617600 database_cache.cc:239]  51 in 0.381s (connected 51)
I20251124 11:11:09.715637 136400352617600 database_cache.cc:250] Building correspondence

✅ Best reconstruction: 2 images registered
DEBUG: Model returned with 2 images.
DEBUG: extracting stairs_split_1_1710453689727.png...
  -> Success!
DEBUG: extracting stairs_split_2_1710453779372.png...
  -> Success!
DEBUG: Added 49 outliers.
Deleted /kaggle/working/outputs/stairs to free up disk space.


In [8]:
if len(results) > 0:
    df = pd.DataFrame(results)
    # Ensure column order matches submission requirements
    df = df[['dataset', 'scene', 'image', 'rotation_matrix', 'translation_vector']]
    df.to_csv("submission.csv", index=False)
    print("✅ submission.csv saved successfully!")
    print(df.head())
    print(df.tail())
else:
    print("⚠️ No results generated (Normal if running in interactive mode without hidden test set).")
    # Create empty dummy to prevent submission error during commit
    with open("submission.csv", "w") as f:
        f.write("dataset,scene,image,rotation_matrix,translation_vector\n")
    print("Created dummy submission.csv")

✅ submission.csv saved successfully!
  dataset   scene                         image  \
0     ETs  scene1  another_et_another_et001.png   
1     ETs  scene1  another_et_another_et002.png   
2     ETs  scene1  another_et_another_et003.png   
3     ETs  scene1  another_et_another_et004.png   
4     ETs  scene1  another_et_another_et005.png   

                                     rotation_matrix  \
0  -0.8977270762569639;0.18290997413556415;0.4007...   
1  -0.9131491279714562;0.13891696187563776;0.3832...   
2  -0.9031469768018159;0.05433613914701933;0.4258...   
3  -0.8724466835953318;0.25588358988558124;0.4163...   
4  -0.848559214760773;0.21783185039600828;0.48217...   

                                  translation_vector  
0  -1.349352477330232;-1.3587726936635953;2.97575...  
1  -1.1800858666061884;-0.5543818006238513;1.0475...  
2  -1.431746790697226;1.1078090236059697;-1.02037...  
3  -1.3199553522332275;-0.8524366363834972;-0.688...  
4  -1.8754318800375427;-1.5691194244744562;0

In [9]:
# Filter for rows where rotation is NOT NaN
successful = df[df['rotation_matrix'].notna()]

print(f"Total Images in CSV: {len(df)}")
print(f"Successfully Registered: {len(successful)}")

print("\n--- Example of Good Data ---")
print(successful.head())

Total Images in CSV: 73
Successfully Registered: 22

--- Example of Good Data ---
  dataset   scene                         image  \
0     ETs  scene1  another_et_another_et001.png   
1     ETs  scene1  another_et_another_et002.png   
2     ETs  scene1  another_et_another_et003.png   
3     ETs  scene1  another_et_another_et004.png   
4     ETs  scene1  another_et_another_et005.png   

                                     rotation_matrix  \
0  -0.8977270762569639;0.18290997413556415;0.4007...   
1  -0.9131491279714562;0.13891696187563776;0.3832...   
2  -0.9031469768018159;0.05433613914701933;0.4258...   
3  -0.8724466835953318;0.25588358988558124;0.4163...   
4  -0.848559214760773;0.21783185039600828;0.48217...   

                                  translation_vector  
0  -1.349352477330232;-1.3587726936635953;2.97575...  
1  -1.1800858666061884;-0.5543818006238513;1.0475...  
2  -1.431746790697226;1.1078090236059697;-1.02037...  
3  -1.3199553522332275;-0.8524366363834972;-0.688...  